In [1]:
from sys import path
path.append('..')
from stationsim.stationsim import Model
from stationsim.ukf import *

In [2]:
"""
width - corridor width
height - corridor height
pop_total -population total
entrances - how many entrances
entrance speed- mean entry speed for agents
exits - how many exits
exit_space- how wide are exits 
speed_min - minimum agents speed to prevent ridiculuous iteration numbers
speed_mean - desired mean of normal distribution of speed of agents
speed_std - as above but standard deviation
speed_steps - how many levels of speed between min and max for each agent
separation - agent radius to determine collisions
wiggle - wiggle distance
batch_iterations - how many model steps to do as a maximum
3 do_ bools for saving plotting and animating data. 
"""
model_params = {
        'pop_total': 10,

        'width': 400,
        'height': 200,

        'gates_in': 3,
        'gates_out': 2,
        'gates_space': 1,
        'gates_speed': 1,

        'speed_min': .2,
        'speed_mean': 1,
        'speed_std': 1,
        'speed_steps': 3,

        'separation': 5,
        'max_wiggle': 1,

        'step_limit': 3600,

        'do_history': True,
        'do_print': True,
    }
"""
Sensor_Noise - how reliable are measurements H_x. lower value implies more reliable
Process_Noise - how reliable is prediction fx lower value implies more reliable
sample_rate - how often to update kalman filter. higher number gives smoother predictions
do_restrict - restrict to a proportion prop of the agents being observed
do_animate - bools for doing animations of agent/wiggle aggregates
do_wiggle_animate
do_density_animate
do_pair_animate
prop - proportion of agents observed. this is a floor function that rounds the proportion 
    DOWN to the nearest intiger number of agents. 1 is all <1/pop_total is none

heatmap_rate - after how many updates to record a frame
bin_size - square sizes for aggregate plots,
do_batch - do batch processing on some pre-recorded truth data.
"""

filter_params = {      

        "Sensor_Noise":  1, 
        "Process_Noise": 1, 
        'sample_rate': 1,
        "do_restrict": True, 
        "do_animate": False,
        "do_wiggle_animate": False,
        "do_density_animate":True,
        "do_pair_animate":False,
        "prop": 0.5,
        "heatmap_rate": 1,
        "bin_size":10,
        "do_batch":False,
        "do_unobserved":False
        }

"""
a - alpha between 1 and 1e-4 typically determines spread of sigma points.
b - beta set to 2 for gaussian. determines trust in prior distribution.
k - kappa usually 0 for state estimation and 3-dim(state) for parameters.
    not 100% sure what kappa does. think its a bias parameter.
!! might be worth making an interactive notebook that varies these. for fun
"""

ukf_params = {

        "a":1,
        "b":2,
        "k":0,
        "d_rate" : 10, #data assimilotion rate every "d_rate model steps recalibrate UKF positions with truth

        }



In [ ]:
"initiate StationSim"
base_model = Model(**model_params)
"initiate UKF"
u = ukf_ss(model_params,filter_params,ukf_params,base_model)
"run UKF main. iterate station sim and track agent locations"
u.main()
actual,preds= u.data_parser(True)

"initiate plot class using final instance of UKF. easier to do this that import numerous variables"
plts = plots(u)

"diagnostic plots, error matrices, and MAE over time"
if filter_params["prop"]<1 or filter_params["do_unobserved"]:
    distances,t_mean = plts.diagnostic_plots(actual,preds,False,False)
distances2,t_mean2 = plts.diagnostic_plots(actual,preds,True,False)



	Iteration: 0/3600
	Iteration: 100/3600
	Iteration: 200/3600
	Iteration: 300/3600


In [ ]:
"further animations"
plts.pair_frames(actual,preds)
plts.heatmap(actual,preds)